<a href="https://colab.research.google.com/github/dm-fedorov/pandas_basic/blob/master/кейсы%20по%20анализу%20данных/Всемирный_банк.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory" target="_blank"></a>

16167 показателей:

In [15]:
import pandas as pd
from pandas_datareader import wb

Извлекаем все индикаторы:

In [2]:
all_ind = wb.get_indicators()

Выводим первые 5 индикаторов:

In [3]:
all_ind.iloc[:5, :2]

,id,name
0,1.0.HCount.1.90usd,Poverty Headcount ($1.90 a day)
1,1.0.HCount.2.5usd,Poverty Headcount ($2.50 a day)
2,1.0.HCount.Mid10to50,Middle Class ($10-50 a day) Headcount
3,1.0.HCount.Ofcl,Official Moderate Poverty Rate-National
4,1.0.HCount.Poor4uds,Poverty Headcount ($4 a day)


Поиск индикаторов, связанных с продолжительностью жизни:

In [4]:
le_ind = wb.search("life expectancy")
le_ind.iloc[:5, :2]

,id,name
10356,SE.SCH.LIFE,"School life expectancy, primary to tertiary, b..."
10357,SE.SCH.LIFE.FE,"School life expectancy, primary to tertiary, f..."
10358,SE.SCH.LIFE.MA,"School life expectancy, primary to tertiary, m..."
11852,SP.DYN.LE00.FE.IN,"Life expectancy at birth, female (years)"
11853,SP.DYN.LE00.IN,"Life expectancy at birth, total (years)"


Каждый индиактор разбит по странам, полный список по странам можно получить:

In [5]:
countries = wb.get_countries()
countries.loc[:5, ["name", "'capitalCity'"]]

,name
0,Aruba
1,Afghanistan
2,Africa
3,Angola
4,Albania
5,Andorra


Получаем данные о продолжительности жизни для всех стран с 1980 по 2019:

In [6]:
le_data_all = wb.download(indicator="SP.DYN.LE00.IN",
                         start="1980",
                         end="2019")
le_data_all

SP.DYN.LE00.IN
country       year                
Canada        2019             NaN
              2018       81.948780
              2017       81.948780
              2016       81.900000
              2015       81.900000
...                            ...
United States 1984       74.563415
              1983       74.463415
              1982       74.360976
              1981       74.009756
              1980       73.609756

[120 rows x 1 columns]

По умолчанию будут возвращены данные только для США, Канады и Мексики:

In [7]:
le_data_all.index.levels[0]

Index(['Canada', 'Mexico', 'United States'], dtype='object', name='country')

Отключаем предупреждения Anaconda:

In [8]:
import warnings
warnings.simplefilter('ignore')

Получаем данные о продолжительности жизни для всех стран с 1980 по 2019 годы:

In [9]:
le_data_all = wb.download(indicator="SP.DYN.LE00.IN", 
                          country = countries['iso2c'],
                          start='1980', 
                          end='2019')
le_data_all

SP.DYN.LE00.IN
country  year                
Aruba    2019             NaN
         2018          76.152
         2017          76.010
         2016          75.868
         2015          75.725
...                       ...
Zimbabwe 1984          61.280
         1983          61.025
         1982          60.650
         1981          60.203
         1980          59.731

[10560 rows x 1 columns]

Преобразуем страны в индекс, а годы - в столбцы:

In [10]:
#le_data_all.pivot(index='country', columns='year')
le_data = le_data_all.reset_index().pivot(index='country', 
                                          columns='year')

Смотрим транспонированные данные:

In [11]:
le_data.iloc[:5,0:3]

SP.DYN.LE00.IN                
year                     1980    1981    1982
country                                      
Afghanistan            43.244  43.923  44.617
Albania                70.208  70.416  70.635
Algeria                58.198  59.519  60.813
American Samoa            NaN     NaN     NaN
Andorra                   NaN     NaN     NaN

Определяем для каждого года страну с наименьшей продолжительностью жизни:

In [12]:
country_with_least_expectancy = le_data.idxmin(axis=0)
country_with_least_expectancy.head()

                year
SP.DYN.LE00.IN  1980       Cambodia
                1981       Cambodia
                1982    Timor-Leste
                1983    South Sudan
                1984    South Sudan
dtype: object

Определяем для каждого года минимальное значение продолжительности жизни:

In [13]:
expectancy_for_least_country = le_data.min(axis=0)
expectancy_for_least_country.head()

                year
SP.DYN.LE00.IN  1980    27.536
                1981    33.342
                1982    38.175
                1983    39.671
                1984    40.005
dtype: float64

Этот программный код объединяет два датафрейма вместе и мы получаем по каждому году страну с наименьшей продолжительностью жизни и наименьшее значение продолжительности жизни:

In [17]:
least = pd.DataFrame(
    data = {'Страна': country_with_least_expectancy.values,
            'Продолжительность': expectancy_for_least_country.values},
    index = country_with_least_expectancy.index.levels[1])
least.head()

,Страна,Продолжительность
year,,
1980,Cambodia,27.536
1981,Cambodia,33.342
1982,Timor-Leste,38.175
1983,South Sudan,39.671
1984,South Sudan,40.005
